In [2]:
#import necessary packages
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import http.client
import json
from pandas import json_normalize

In [45]:
#import Data

#import PL data
PL_18_19 = pd.read_csv('Data\E_18_19.csv')
PL_19_20 = pd.read_csv('Data\E_19_20.csv')
PL_20_21 = pd.read_csv('Data\E_20_21.csv')
PL_21_22 = pd.read_csv('Data\E_21_22.csv')

#import xG for PL
PL_18_19_xG = pd.read_csv('Data\E_18_19_xG.csv')
PL_19_20_xG = pd.read_csv('Data\E_19_20_xG.csv')
PL_20_21_xG = pd.read_csv('Data\E_20_21_xG.csv')
PL_21_22_xG = pd.read_csv('Data\E_21_22_xG.csv')


,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report
0,1.0,Fri,2021-08-13,20:00 (21:00),Brentford,1.3,2–0,1.4,Arsenal,16479.0,Brentford Community Stadium,Michael Oliver,Match Report
1,1.0,Sat,2021-08-14,12:30 (13:30),Manchester Utd,1.5,5–1,0.6,Leeds United,72732.0,Old Trafford,Paul Tierney,Match Report
2,1.0,Sat,2021-08-14,15:00 (16:00),Leicester City,0.8,1–0,1.1,Wolves,31983.0,King Power Stadium,Craig Pawson,Match Report
3,1.0,Sat,2021-08-14,15:00 (16:00),Burnley,1.0,1–2,1.3,Brighton,16910.0,Turf Moor,David Coote,Match Report
4,1.0,Sat,2021-08-14,15:00 (16:00),Chelsea,0.9,3–0,0.3,Crystal Palace,38965.0,Stamford Bridge,Jonathan Moss,Match Report
...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,38.0,Sun,2022-05-22,16:00 (17:00),Norwich City,NaN,NaN,NaN,Tottenham,NaN,Carrow Road,NaN,Head-to-Head
435,38.0,Sun,2022-05-22,16:00 (17:00),Manchester City,NaN,NaN,NaN,Aston Villa,NaN,Etihad Stadium,NaN,Head-to-Head
436,38.0,Sun,2022-05-22,16:00 (17:00),Liverpool,NaN,NaN,NaN,Wolves,NaN,Anfield,NaN,Head-to-Head
437,38.0,Sun,2022-05-22,16:00 (17:00),Burnley,NaN,NaN,NaN,Newcastle Utd,NaN,Turf Moor,NaN,Head-to-Head


In [46]:
#Merge data
PL_total = pd.concat([PL_18_19, PL_19_20, PL_20_21, PL_21_22])
PL_total_xG = pd.concat([PL_18_19_xG, PL_19_20_xG, PL_20_21_xG, PL_21_22_xG])

#reset index
PL_total.reset_index(inplace=False, drop=True)

#remove nan values and reset index
PL_total_xG.drop(labels='Notes', axis=1, inplace=True)
PL_total_xG.dropna(axis=0, inplace=True, how='all')
PL_total_xG.reset_index(inplace=True, drop=True)


PL_total_xG

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report
0,1.0,Fri,2018-08-10,20:00 (21:00),Manchester Utd,1.7,2–1,1.0,Leicester City,74439.0,Old Trafford,Andre Marriner,Match Report
1,1.0,Sat,2018-08-11,12:30 (13:30),Newcastle Utd,0.9,1–2,2.0,Tottenham,51749.0,St. James' Park,Martin Atkinson,Match Report
2,1.0,Sat,2018-08-11,15:00 (16:00),Fulham,0.6,0–2,1.0,Crystal Palace,24821.0,Craven Cottage,Mike Dean,Match Report
3,1.0,Sat,2018-08-11,15:00 (16:00),Bournemouth,2.1,2–0,1.2,Cardiff City,10353.0,Vitality Stadium,Kevin Friend,Match Report
4,1.0,Sat,2018-08-11,15:00 (16:00),Watford,1.6,2–0,0.2,Brighton,20051.0,Vicarage Road Stadium,Jonathan Moss,Match Report
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,38.0,Sun,2022-05-22,16:00 (17:00),Norwich City,NaN,NaN,NaN,Tottenham,NaN,Carrow Road,NaN,Head-to-Head
1516,38.0,Sun,2022-05-22,16:00 (17:00),Manchester City,NaN,NaN,NaN,Aston Villa,NaN,Etihad Stadium,NaN,Head-to-Head
1517,38.0,Sun,2022-05-22,16:00 (17:00),Liverpool,NaN,NaN,NaN,Wolves,NaN,Anfield,NaN,Head-to-Head
1518,38.0,Sun,2022-05-22,16:00 (17:00),Burnley,NaN,NaN,NaN,Newcastle Utd,NaN,Turf Moor,NaN,Head-to-Head


In [39]:
#API Import


conn = http.client.HTTPSConnection("api-football-v1.p.rapidapi.com")

headers = {
    'x-rapidapi-host': "api-football-v1.p.rapidapi.com",
    'x-rapidapi-key': "b8bd6a409emshdd09020617fc3afp1d89f3jsnf6eb60efca6d"
    }

conn.request("GET", "/v3/fixtures/statistics?fixture=2755", headers=headers)

res = conn.getresponse()
data = res.read()

#res.encoding 'utf-8'
out = json.loads(data)
#data.decode("utf-8")
print (json.dumps(out, indent = 4, sort_keys=True))
#print(out)

{
    "errors": [],
    "get": "fixtures/statistics",
    "paging": {
        "current": 1,
        "total": 1
    },
    "parameters": {
        "fixture": "2755"
    },
    "response": [
        {
            "statistics": [
                {
                    "type": "Shots on Goal",
                    "value": 3
                },
                {
                    "type": "Shots off Goal",
                    "value": 6
                },
                {
                    "type": "Total Shots",
                    "value": 15
                },
                {
                    "type": "Blocked Shots",
                    "value": 6
                },
                {
                    "type": "Shots insidebox",
                    "value": 8
                },
                {
                    "type": "Shots outsidebox",
                    "value": 7
                },
                {
                    "type": "Fouls",
                    "value": 18
   